<a href="https://colab.research.google.com/github/benjuarez8/CS81C-Research-Bolivia/blob/main/cs81c_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
pip install Unidecode

     |████████████████████████████████| 235 kB 7.4 MB/s 


In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import csv
from xgboost.sklearn import XGBClassifier
from unidecode import unidecode

#Load Data

In [3]:
training_data = pd.read_csv("https://raw.githubusercontent.com/benjuarez8/Bolivia-SURF-2020/master/CS81C/training_data.csv")
training_data.rename( columns={"Unnamed: 0":"ID"}, inplace=True )

In [4]:
testing_data = pd.read_csv("https://raw.githubusercontent.com/benjuarez8/Bolivia-SURF-2020/master/CS81C/testing_data.csv")
testing_data.rename( columns={"Unnamed: 0":"ID"}, inplace=True )

#Data Processing

In [5]:
training_data["Fraud_num"] = [ -1 if type == "VS" else 1 if type == "BBS" else 0 for type in training_data["Fraud.type"] ]

In [6]:
training_departments = training_data["Department"].str.get_dummies()
training_data = pd.concat([training_data, training_departments], axis=1)
testing_departments = testing_data["Department"].str.get_dummies()
testing_data = pd.concat([testing_data, testing_departments], axis=1)
testing_data.rename( columns={"Potosí":unidecode("Potosí")}, inplace=True)

In [7]:
train_labels = training_data["Fraud_num"]
train = training_data[["Beni", "Chuquisaca", "Cochabamba", "La Paz", "Oruro", "Pando", "Potosi", "Santa Cruz", "Tarija", "Turnout", "MAS.vote"]]
test = testing_data[["Beni", "Chuquisaca", "Cochabamba", "La Paz", "Oruro", "Pando", "Potosi", "Santa Cruz", "Tarija", "Turnout", "MAS.vote"]]

In [8]:
X_train = np.array(train)
y_train = np.array(train_labels)
X_test = np.array(test)

#XGBoost

In [9]:
seed = 8
model = XGBClassifier()
model.fit(X_train, y_train)
print(model)

XGBClassifier(objective='multi:softprob')


In [10]:
y_pred = model.predict(X_test)

In [11]:
test["Pred_Fraud_num"] = y_pred.tolist()
test["Pred_Fraud_type"] = [ "VS" if type == -1 else "BBS" if type == 1 else "Clean" for type in test["Pred_Fraud_num"] ]
test["Department"] = testing_data["Department"]
test["Precinct"] = testing_data["Precinct"]
test["Municipality"] = testing_data["Municipality"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [12]:
test.to_csv("xgboost_results.csv", index=False)

#Results

In [13]:
col_names = ["Clean", "At_Risk", "BBS_Risk", "VS_Risk", "Avg_Turnout", "Avg_MAS_vote"]
results = pd.DataFrame(columns = col_names)
counts = test.Pred_Fraud_type.value_counts()
clean = counts[0] / (counts[0] + counts[1] + counts[2])
tot_risk = (counts[1] + counts[2]) / (counts[0] + counts[1] + counts[2])
bbs = counts[1] / (counts[0] + counts[1] + counts[2])
vs = counts[2] / (counts[0] + counts[1] + counts[2])
avg_turnout = test["Turnout"].mean()
avg_vote = test["MAS.vote"].mean()
results.loc[len(results)] = [round(100*clean,2), round(100*tot_risk,2), round(100*bbs,2), round(100*vs,2), round(100*avg_turnout,2), round(avg_vote,2)]

In [14]:
results.to_csv("xgboost_tot_results.csv", index=False)
results

,Clean,At_Risk,BBS_Risk,VS_Risk,Avg_Turnout,Avg_MAS_vote
0,85.92,14.08,9.5,4.58,89.8,0.46


In [15]:
col_names = ["Department", "Clean", "At_Risk", "BBS_Risk", "VS_Risk", "Avg_Turnout", "Avg_MAS_vote"]
xgb_dep_results = pd.DataFrame(columns = col_names)
for dep in test["Department"].unique():
    vars()[dep] = test[test["Department"] == dep].Pred_Fraud_type.value_counts()
    counts = test[test["Department"] == dep].Pred_Fraud_type.value_counts()
    clean = counts[0] / (counts[0] + counts[1] + counts[2])
    tot_risk = (counts[1] + counts[2]) / (counts[0] + counts[1] + counts[2])
    bbs = counts[1] / (counts[0] + counts[1] + counts[2])
    vs = counts[2] / (counts[0] + counts[1] + counts[2])
    avg_turnout = test[test["Department"] == dep]["Turnout"].mean()
    avg_vote = test[test["Department"] == dep]["MAS.vote"].mean()
    xgb_dep_results.loc[len(xgb_dep_results)] = [dep, round(100*clean,2), round(100*tot_risk,2), round(100*bbs,2), round(100*vs,2), round(100*avg_turnout,2), round(avg_vote,2)]
xgb_dep_results = xgb_dep_results.sort_values(by = "Clean")
xgb_dep_results.reset_index(drop=True, inplace=True)

In [16]:
xgb_dep_results.to_csv("xgboost_dep_results.csv", index=False)
xgb_dep_results

,Department,Clean,At_Risk,BBS_Risk,VS_Risk,Avg_Turnout,Avg_MAS_vote
0,Cochabamba,75.61,24.39,13.20,11.19,90.28,0.57
1,La Paz,83.94,16.06,12.88,3.18,91.75,0.53
2,Potosí,83.99,16.01,8.86,7.15,88.87,0.46
3,Oruro,84.40,15.60,13.10,2.50,91.09,0.47
4,Chuquisaca,91.16,8.84,5.89,2.95,88.41,0.40
5,Santa Cruz,91.55,8.45,7.02,1.42,88.83,0.34
6,Tarija,94.89,5.11,2.72,2.39,87.26,0.39
7,Pando,95.42,4.58,3.05,1.53,84.31,0.43
8,Beni,96.46,3.54,2.14,1.40,86.17,0.34


In [17]:
col_names = ["Municipality", "Department", "Clean", "At_Risk", "BBS_Risk", "VS_Risk", "Avg_Turnout", "Avg_MAS_vote"]
xgb_municip_results = pd.DataFrame(columns = col_names)
for municip in test["Municipality"].unique():
  dep = test.loc[test["Municipality"] == municip]["Department"].iloc[0]
  counts = test[test["Municipality"] == municip].Pred_Fraud_type.value_counts()
  if (len(counts) > 2):
    clean = counts[0] / (counts[0] + counts[1] + counts[2])
    tot_risk = (counts[1] + counts[2]) / (counts[0] + counts[1] + counts[2])
    bbs = counts[1] / (counts[0] + counts[1] + counts[2])
    vs = counts[2] / (counts[0] + counts[1] + counts[2])
    avg_turnout = test[test["Municipality"] == municip]["Turnout"].mean()
    avg_vote = test[test["Municipality"] == municip]["MAS.vote"].mean()
    xgb_municip_results.loc[len(xgb_municip_results)] = [municip, dep, round(100*clean,2), round(100*tot_risk,2), round(100*bbs,2), round(100*vs,2), round(100*avg_turnout,2), round(avg_vote,2)]
xgb_municip_results = xgb_municip_results.sort_values(by = "Clean")
xgb_municip_results.reset_index(drop=True, inplace=True)

In [18]:
xgb_municip_results.to_csv("xgboost_municip_results.csv", index=False)
xgb_municip_results

,Municipality,Department,Clean,At_Risk,BBS_Risk,VS_Risk,Avg_Turnout,Avg_MAS_vote
0,Chacarilla,La Paz,33.33,66.67,33.33,33.33,90.33,0.81
1,Shinahota,Cochabamba,35.94,64.06,34.38,29.69,91.33,0.86
2,Entre Ríos,Cochabamba,39.61,60.39,30.52,29.87,91.46,0.78
3,Pocoata,Potosí,42.86,57.14,34.29,22.86,88.96,0.81
4,Pojo,Cochabamba,42.86,57.14,33.33,23.81,92.67,0.88
...,...,...,...,...,...,...,...,...
125,Villazón,Potosí,95.45,4.55,3.64,0.91,82.38,0.65
126,Tupiza,Potosí,95.45,4.55,3.64,0.91,87.35,0.56
127,Tarija,Tarija,96.18,3.82,3.55,0.27,88.68,0.31
128,Montero,Santa Cruz,96.69,3.31,2.21,1.10,88.64,0.39
